In [36]:
import numpy as np
import cvxpy as cp
from basis_generator import rand_moment, sel_seq, proj_mul
import json

In [2]:
def single_behavior_visibility(X, X_basis):
    
    eta = cp.Variable((1, 1))
    alpha = cp.Variable((len(X_basis), 1))
    beta = cp.Variable((len(X_basis), 1))
    M = cp.Variable(X_basis[0].shape)
    N = cp.Variable(X_basis[0].shape)

    constraints = [N >> 0]
    constraints += [N == sum([beta[j]*X_basis[j] for j in range(len(X_basis))])]
    constraints += [N[0,0] == 1 - eta]

    for i in range(1,len(X)):
        constraints += [
            eta*X[i,i] + N[i,i] == M[i,i]
        ]

    constraints += [M >> 0]
    constraints += [M == sum([alpha[j]*X_basis[j] for j in range(len(X_basis))])]
    constraints += [M[0,0] == 1]

    prob = cp.Problem(cp.Maximize(eta),
                      constraints)
    prob.solve(solver=cp.MOSEK, verbose=False)
    
    coef = []
    for __ in range(3,len(constraints)-3):
        coef.append(np.real(constraints[__].dual_value[0][0]))
    
    return eta.value[0][0], coef

In [43]:
def NPA_bound(coef,X_basis):
    beta = cp.Variable((len(X_basis), 1))
    N = cp.Variable(X_basis[0].shape)

    constraints = [N >> 0]
    constraints += [N == sum([beta[j]*X_basis[j] for j in range(len(X_basis))])]
    constraints += [N[0,0] == 1]

    prob = cp.Problem(cp.Minimize(
            sum([N[j+1,j+1]*coef[j] for j in range(len(coef))])
            ),constraints)
    prob.solve(solver=cp.MOSEK, verbose=False)
    return prob.value

In [45]:
# Import basis
basis_filename = "data_basis/3-dim-2-num_obs-3-len_seq-3-num_out.npy"
X_basis = np.load(basis_filename)

FileNotFoundError: [Errno 2] No such file or directory: 'data_basis/3-dim-2-num_obs-3-len_seq-3-num_out.npy'

In [2]:
dimX = 4
dim_base = 3
num_obs = 2
len_seq = 3
num_out = 3
remove_last_out = True

In [6]:
X, rho, P = rand_moment(dimX, num_obs, len_seq, num_out, [len_seq], remove_last_out)
eta, coef = single_behavior_visibility(X, X_basis)

In [80]:
for __ in range(5):
    X_t, rho_t, P_t = rand_moment(dimX, num_obs, len_seq, num_out, [len_seq], remove_last_out)
    eta_t, coef_t = single_behavior_visibility(X_t, X_basis)
    if eta_t < eta:
        eta = eta_t
        coef = coef_t
        X = X_t
        rho = rho_t
        P = P_t

print(eta)

0.8201995428547537


In [81]:
Q = np.real(np.diag(X)[1:] @ coef)
print(Q)

-0.7488070170722095


In [82]:
C = NPA_bound(coef,X_basis)
print(C)

-0.5690064392946877


In [83]:
data = {}
# data["Moment Matrix"] = X
# data["Quantum State"] = rho
# data["Measurements"] = P
data["Inequality"] = coef
data["Classical Bound"] = C
data["Quantum Bound"] = Q

data["num of observables"] = num_obs
data["maximum length of sequences"] = len_seq
data["num of outcomes"] = num_out
data["dimension behavior"] = dimX
data["dimension base"] = dim_base

NAME = '{}-num_obs-{}-len_seq-{}-out_max-{}-dim_behavior-{}-dim_base'.format(num_obs, len_seq, num_out, dimX, dim_base)

with open("dimension_witness/" + NAME + '.json', 'w') as fp:
    json.dump(data, fp, indent=2)
    
np.save("dimension_witness/" + NAME + "-moment_matrix", X)
np.save("dimension_witness/" + NAME + "-state", rho)
np.save("dimension_witness/" + NAME + "-measurements", P)

In [1]:
"dimension_witness/" + NAME + "-state"

NameError: name 'NAME' is not defined

## Verification

In [3]:
NAME = '{}-num_obs-{}-len_seq-{}-out_max-{}-dim_behavior-{}-dim_base'.format(num_obs, len_seq, num_out, dimX, dim_base)

In [11]:
#### Loading and testing state
rho = np.load("dimension_witness/" + NAME + "-state.npy")
print(rho.shape)
np.trace(rho @ rho)

(4, 4)


(0.9999999999999994+0j)

In [13]:
#### Loading measurements
P = np.load("dimension_witness/" + NAME + "-measurements.npy")
print(P.shape)
np.trace(rho @ rho)

(2, 3, 4, 4)


(0.9999999999999994+0j)

In [27]:
#### Completness
P[0][0]+P[0][1]+P[0][2]

array([[ 1.00000000e+00+0.00000000e+00j, -9.02056208e-17+6.93889390e-18j,
         4.85722573e-17+5.55111512e-17j, -1.11022302e-16+1.66533454e-16j],
       [-9.02056208e-17-6.93889390e-18j,  1.00000000e+00+0.00000000e+00j,
         3.46944695e-18-2.51534904e-16j, -8.32667268e-17+6.93889390e-18j],
       [ 4.85722573e-17-5.55111512e-17j,  3.46944695e-18+2.51534904e-16j,
         1.00000000e+00+0.00000000e+00j, -6.93889390e-17-1.38777878e-17j],
       [-1.11022302e-16-1.66533454e-16j, -8.32667268e-17-6.93889390e-18j,
        -6.93889390e-17+1.38777878e-17j,  1.00000000e+00+0.00000000e+00j]])

In [28]:
#### Completness
P[1][0]+P[1][1]+P[1][2]

array([[ 1.00000000e+00+0.00000000e+00j,  4.02455846e-16+6.93889390e-17j,
         0.00000000e+00-1.66533454e-16j,  5.55111512e-17-6.24500451e-17j],
       [ 4.02455846e-16-6.93889390e-17j,  1.00000000e+00+0.00000000e+00j,
         8.32667268e-17-5.55111512e-17j, -7.28583860e-17+1.11022302e-16j],
       [ 0.00000000e+00+1.66533454e-16j,  8.32667268e-17+5.55111512e-17j,
         1.00000000e+00+0.00000000e+00j,  5.55111512e-17-2.77555756e-17j],
       [ 5.55111512e-17+6.24500451e-17j, -7.28583860e-17-1.11022302e-16j,
         5.55111512e-17+2.77555756e-17j,  1.00000000e+00+0.00000000e+00j]])

In [29]:
P[0][0]

array([[ 0.55914889+0.j        ,  0.10841001-0.02721568j,
        -0.15703023-0.00731945j, -0.19230265-0.41510915j],
       [ 0.10841001+0.02721568j,  0.02234364+0.j        ,
        -0.03008938-0.00906232j, -0.01707963-0.08984304j],
       [-0.15703023+0.00731945j, -0.03008938+0.00906232j,
         0.04419586+0.j        ,  0.0594398 +0.1140611j ],
       [-0.19230265+0.41510915j, -0.01707963+0.08984304j,
         0.0594398 -0.1140611j ,  0.3743116 +0.j        ]])

In [30]:
P[0][0]@P[0][0]

array([[ 0.55914889+0.j        ,  0.10841001-0.02721568j,
        -0.15703023-0.00731945j, -0.19230265-0.41510915j],
       [ 0.10841001+0.02721568j,  0.02234364+0.j        ,
        -0.03008938-0.00906232j, -0.01707963-0.08984304j],
       [-0.15703023+0.00731945j, -0.03008938+0.00906232j,
         0.04419586+0.j        ,  0.0594398 +0.1140611j ],
       [-0.19230265+0.41510915j, -0.01707963+0.08984304j,
         0.0594398 -0.1140611j ,  0.3743116 +0.j        ]])

In [34]:
sequences = sel_seq(num_obs, len_seq, num_out, remove_last_out=True)

In [37]:
X = np.eye(len(sequences)+1, dtype=complex)
for i, seq_row in enumerate(sequences):
    Pi = proj_mul([P[k] for k in seq_row[1]], seq_row[0])
    #X[0,i+1] = np.trace(Pi @ np.eye(dim) @ rho)
    #X[i+1,0] = np.conjugate(X[0,i+1])#np.trace(np.eye(dim) @ np.conjugate(Pi.T) @ rho)
    for j, seq_col in enumerate(sequences):
        if i==j:
            Pj= proj_mul([P[k] for k in seq_col[1]], seq_col[0])
            X[i+1,j+1] = np.trace(Pi @ np.conjugate(Pj.T) @ rho)

In [40]:
with open("dimension_witness/" + NAME + '.json') as json_file:
     ineq = json.load(json_file)

In [41]:
np.real(np.diag(X)[1:] @ ineq["Inequality"])

-0.7488070170722096

In [44]:
NPA_bound(ineq["Inequality"],X_basis)

NameError: name 'X_basis' is not defined